In [2]:
# Classic imports
import pandas as pd
import numpy as np

# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
# - Models -
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

# Other imports

In [4]:
# Load the datasets
train_name = "train_with_clusters_with_sin_ratios_with_m2_price_with_zipcode_average_price_scaled_standard" # nom du fichier csv à utiliser pour train
pred_name = "test_with_clusters_with_sin_ratios_with_m2_price_with_zipcode_average_price_scaled_standard" # nom du fichier csv pour prédire

train_df = pd.read_csv('CSV DATA/'+ train_name +'.csv')  # fichier csv à utiliser pour train
pred_df = pd.read_csv('CSV DATA/'+ pred_name +'.csv')  # fichier csv pour prédire

target_columns = ['prix', 'prix_m2_interieur' , 'prix_m2_total'] # colonnes à prédire

#little printy print to see the data
print("Train shape: ", train_df.shape)
print("Prediction file shape: ", pred_df.shape)

Train shape:  (17147, 32)
Prediction file shape:  (4287, 29)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(columns=target_columns), train_df[target_columns], test_size=0.2, random_state=42)

In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13717, 29), (3430, 29), (13717, 3), (3430, 3))

In [7]:
def cross_val_func(model, X, y, cv=5):
    # Perform cross-validation
    scores = cross_val_score(model, X, y, cv=cv)  # 5-fold cross-validation

    # Print the cross-validation scores
    print(f"Cross-validation scores: {scores}")
    print(f"Mean cross-validation score: {scores.mean()}")
    print(f"Standard deviation of scores: {scores.std()}")

    return scores

In [8]:
def RMSE_calc(x1, x2):
    RMSE = np.sqrt(mean_squared_error(x1, x2))
    print("RMSE: ", RMSE)
    return RMSE

In [9]:
from sklearn.linear_model import Lasso
def lasso_reg(X_train, y_train, X_test, lasso = None):
    if lasso :
        lasso = lasso
    else :
        # Initialisation du modèle Lasso
        lasso = Lasso(alpha=0.1)

    # Entraînement du modèle
    lasso.fit(X_train, y_train)

    # Prédiction sur l'ensemble de test
    y_pred = lasso.predict(X_test)

    return y_pred, lasso

In [10]:
lasso = Lasso(alpha=0.1)
y_pred_lasso = lasso_reg(X_train, y_train, X_test, lasso)[0]
scores_crossval_lasso = cross_val_func(lasso, X_train, y_train, cv = 5)
RMSE_lasso = RMSE_calc(y_test, y_pred_lasso)


d:\M2 - Ponts\PRAMA\prama-2025\PRAMA2025\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.905e+13, tolerance: 1.894e+11
  model = cd_fast.enet_coordinate_descent(
d:\M2 - Ponts\PRAMA\prama-2025\PRAMA2025\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.311e+07, tolerance: 1.950e+06
  model = cd_fast.enet_coordinate_descent(
d:\M2 - Ponts\PRAMA\prama-2025\PRAMA2025\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

Cross-validation scores: [0.73164823 0.70945852 0.7275713  0.72245201 0.70608765]
Mean cross-validation score: 0.7194435451055067
Standard deviation of scores: 0.010021440790805667
RMSE:  95660.65086809943


d:\M2 - Ponts\PRAMA\prama-2025\PRAMA2025\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.090e+06, tolerance: 2.694e+05
  model = cd_fast.enet_coordinate_descent(
